In [29]:
import os
import json
import random
from tqdm import tqdm

import esper.face_embeddings as face_embed
from esper.widget import *
from esper.prelude import *

OUTPUT_DIR = '/app/data/azure-face-crops/'

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [18]:
VIDEO_NAMES = [
    'CNNW_20131214_160000_CNN_Newsroom', 
    'FOXNEWSW_20161001_160000_Americas_Election_HQ',
    'MSNBCW_20170701_060000_The_11th_Hour_With_Brian_Williams'
]

In [19]:
def cluster_for_video(video_name, k=100):
    face_ids = [
        f['id'] for f in Face.objects.filter(
            frame__video__path__contains=video_name).values('id')]
    clusters = [[] for _ in range(k)]
    for f, c in face_embed.kmeans(face_ids, k):
        clusters[c].append(f)
    clusters.sort(key=lambda x: -len(x))
    return clusters

In [ ]:
results = []
for cluster in cluster_for_video(VIDEO_NAMES[0]):
    results.append(qs_to_result(Face.objects.filter(id__in=cluster), limit=3))
esper_widget(group_results(list(enumerate(results))), 
             disable_captions=True, jupyter_keybindings=True)

In [30]:
def crop_bbox(img, bbox, expand=0.1):
    y1 = max(bbox.bbox_y1 - expand, 0)
    y2 = min(bbox.bbox_y2 + expand, 1)
    x1 = max(bbox.bbox_x1 - expand, 0)
    x2 = min(bbox.bbox_x2 + expand, 1)
    [h, w] = img.shape[:2]
    return img[int(y1 * h):int(y2 * h), int(x1 * w):int(x2 * w)]


def write_result_for_video(video_name, clusters, n_samples=5):
    outdir = os.path.join(OUTPUT_DIR, video_name)
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    with open(os.path.join(outdir, 'clusters.json'), 'w') as f:
        json.dump(clusters, f)
    face_ids = [
        i for c in clusters for i in random.sample(c, min(n_samples, len(c)))]
    print('Sampling {} faces from {}'.format(len(face_ids), video_name))
    for face in tqdm(Face.objects.filter(id__in=face_ids).select_related()):
        img = crop_bbox(load_frame(face.frame.video, face.frame.number, []), face)
        cv2.imwrite(os.path.join(outdir, '{}.png'.format(face.id)), img)

In [31]:
for video_name in VIDEO_NAMES:
    write_result_for_video(video_name, cluster_for_video(video_name))

Sampling 484 faces from CNNW_20131214_160000_CNN_Newsroom



Sampling 496 faces from FOXNEWSW_20161001_160000_Americas_Election_HQ



Sampling 475 faces from MSNBCW_20170701_060000_The_11th_Hour_With_Brian_Williams
